#DCGAN Task

Essentially given an input data x:
* The Generator G(z,Θ) is a mapping from some input distribution to $\rho_g \sim x$
* The Discriminator D(z,Θ) is a mapping from $ρ_g$ to [0,1] which gives us the probability of whether z is a part of original distribution x  

* here our x is Training imgs
* $\rho_g$ is generated imgs





#Importing Libs

In [ ]:
import numpy as np
import math

import torchvision.transforms as transforms
from torchvision.utils import save_image

from torchvision import datasets
from torch.autograd import Variable

import torch.nn as nn
import torch.nn.functional as F
import torch

#Defining the hyperparameters/input shape etc. as a class to access later

I have changed the hyperparams for faster training (may not be the best accuracy)

In [ ]:
os.makedirs("images", exist_ok=True)

class OPT:
  def __init__(self,epochs,batch,lr,b1,b2,cpu,lat,img,chan,samp):
    self.n_epochs = epochs
    self.batch_size = batch
    self.lr = lr
    self.b1 = b1
    self.b2 = b2
    self.n_cpu = cpu
    self.latent_dim = lat
    self.img_size = img
    self.channels = chan
    self.sample_interval = samp

opt = OPT(50,1024,0.0002,0.5,0.999,4,100,32,1,400)

cuda = True if torch.cuda.is_available() else False

#init weights for conv and batchnorm layers with std : 0.02 and mean 0/1 for conv and batch norm resp.

This helps the optimizer to not get s

In [ ]:
def weights_init_normal(m):
    classname = m.__class__.__name__
    if classname.find("Conv") != -1:
        torch.nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find("BatchNorm2d") != -1:
        torch.nn.init.normal_(m.weight.data, 1.0, 0.02)
        torch.nn.init.constant_(m.bias.data, 0.0)

#Defining the model as given in the paper

Some Intresting things to note
*   We do not have a FC layer like a normal CNN, we have a batchnorm layer which helps with the latency of inference and time of training
*   It uses LeakyRelu which helps with the dying ReLU problem
*   Uses Droput layers to increase the robustness of the model


In [ ]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()

        self.init_size = opt.img_size // 4
        self.l1 = nn.Sequential(nn.Linear(opt.latent_dim, 128 * self.init_size ** 2))

        self.conv_blocks = nn.Sequential(
            nn.BatchNorm2d(128),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(128, 128, 3, stride=1, padding=1),
            nn.BatchNorm2d(128, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(128, 64, 3, stride=1, padding=1),
            nn.BatchNorm2d(64, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64, opt.channels, 3, stride=1, padding=1),
            nn.Tanh(),
        )

    def forward(self, z):
        out = self.l1(z)
        out = out.view(out.shape[0], 128, self.init_size, self.init_size)
        img = self.conv_blocks(out)
        return img

In [ ]:

class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        def discriminator_block(in_filters, out_filters, bn=True):
            block = [nn.Conv2d(in_filters, out_filters, 3, 2, 1), nn.LeakyReLU(0.2, inplace=True), nn.Dropout2d(0.25)]
            if bn:
                block.append(nn.BatchNorm2d(out_filters, 0.8))
            return block

        self.model = nn.Sequential(
            *discriminator_block(opt.channels, 16, bn=False),
            *discriminator_block(16, 32),
            *discriminator_block(32, 64),
            *discriminator_block(64, 128),
        )
        ds_size = opt.img_size // 2 ** 4
        self.adv_layer = nn.Sequential(nn.Linear(128 * ds_size ** 2, 1), nn.Sigmoid())

    def forward(self, img):
        out = self.model(img)
        out = out.view(out.shape[0], -1)
        validity = self.adv_layer(out)

        return validity


In [ ]:
# Loss function
adversarial_loss = torch.nn.BCELoss()

In [ ]:
# Initialising generator and discriminator
generator = Generator()
discriminator = Discriminator()

In [ ]:
#using gpu for discriminator and generator
if cuda:
    generator.cuda()
    discriminator.cuda()
    adversarial_loss.cuda()

In [ ]:
cuda

True

In [ ]:
#Applying normal distribution as given in paper
generator.apply(weights_init_normal)
discriminator.apply(weights_init_normal)

Discriminator(
  (model): Sequential(
    (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Dropout2d(p=0.25, inplace=False)
    (3): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (4): LeakyReLU(negative_slope=0.2, inplace=True)
    (5): Dropout2d(p=0.25, inplace=False)
    (6): BatchNorm2d(32, eps=0.8, momentum=0.1, affine=True, track_running_stats=True)
    (7): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (8): LeakyReLU(negative_slope=0.2, inplace=True)
    (9): Dropout2d(p=0.25, inplace=False)
    (10): BatchNorm2d(64, eps=0.8, momentum=0.1, affine=True, track_running_stats=True)
    (11): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (12): LeakyReLU(negative_slope=0.2, inplace=True)
    (13): Dropout2d(p=0.25, inplace=False)
    (14): BatchNorm2d(128, eps=0.8, momentum=0.1, affine=True, track_running_stats=True)
  )
  (adv_laye

#Downloading the Dataset : MNIST Numbers

This was in the code, thought I will just use this

In [ ]:
os.makedirs("../../data/mnist", exist_ok=True)
dataloader = torch.utils.data.DataLoader(
    datasets.MNIST(
        "../../data/mnist",
        train=True,
        download=True,
        transform=transforms.Compose(
            [transforms.Resize(opt.img_size), transforms.ToTensor(), transforms.Normalize([0.5], [0.5])]
        ),
    ),
    batch_size=opt.batch_size,
    shuffle=True,
)


100%|██████████| 9912422/9912422 [00:00<00:00, 449890288.64it/s]

Extracting ../../data/mnist/MNIST/raw/train-images-idx3-ubyte.gz to ../../data/mnist/MNIST/raw


100%|██████████| 28881/28881 [00:00<00:00, 38321953.12it/s]


Extracting ../../data/mnist/MNIST/raw/train-labels-idx1-ubyte.gz to ../../data/mnist/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 216724370.80it/s]

Extracting ../../data/mnist/MNIST/raw/t10k-images-idx3-ubyte.gz to ../../data/mnist/MNIST/raw




100%|██████████| 4542/4542 [00:00<00:00, 3082609.83it/s]


Extracting ../../data/mnist/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../../data/mnist/MNIST/raw



#Setting opitimizer and alias for tensor class

The paper seems to have used Mini-Batch SGD, I would prefer Adam

In [ ]:
#using adam opitmizer
optimizer_G = torch.optim.Adam(generator.parameters(), lr=opt.lr, betas=(opt.b1, opt.b2))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=opt.lr, betas=(opt.b1, opt.b2))

Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor


#Training Loop

* We init the gnd truths for the gen and real imgs
* then we train the gen by feeding in gaussian noise and real imgs
* we train the discriminator on the real and fake imgs
* One thing to note is that the real_imgs is a leaf variable thus the loss info from gen are carried to disc and vice-versa

In [ ]:
for epoch in range(opt.n_epochs):
    for i, (imgs, _) in enumerate(dataloader):

        # Init adversarial ground truths
        valid = Variable(Tensor(imgs.shape[0], 1).fill_(1.0), requires_grad=False)
        fake = Variable(Tensor(imgs.shape[0], 1).fill_(0.0), requires_grad=False)
        real_imgs = Variable(imgs.type(Tensor))


        #generator loop
        optimizer_G.zero_grad()
        z = Variable(Tensor(np.random.normal(0, 1, (imgs.shape[0], opt.latent_dim))))
        gen_imgs = generator(z)
        g_loss = adversarial_loss(discriminator(gen_imgs), valid)
        g_loss.backward()
        optimizer_G.step()


        #discriminator loop
        optimizer_D.zero_grad()
        real_loss = adversarial_loss(discriminator(real_imgs), valid)
        fake_loss = adversarial_loss(discriminator(gen_imgs.detach()), fake)
        d_loss = (real_loss + fake_loss) / 2
        d_loss.backward()
        optimizer_D.step()

        print(
            f"[Epoch {epoch}{opt.n_epochs}] [Batch {i}{len(dataloader)}] [D loss: {d_loss.item()}] [G loss: %f]"
            % (epoch, opt.n_epochs, i, len(dataloader), d_loss.item(), g_loss.item())
        )

        batches_done = epoch * len(dataloader) + i
        if batches_done % opt.sample_interval == 0:
            save_image(gen_imgs.data[:25], "images/%d.png" % batches_done, nrow=5, normalize=True)

[Epoch 0/50] [Batch 0/59] [D loss: 0.693352] [G loss: 0.672998]
[Epoch 0/50] [Batch 1/59] [D loss: 0.693257] [G loss: 0.673688]
[Epoch 0/50] [Batch 2/59] [D loss: 0.693147] [G loss: 0.674437]
[Epoch 0/50] [Batch 3/59] [D loss: 0.693044] [G loss: 0.675041]
[Epoch 0/50] [Batch 4/59] [D loss: 0.692917] [G loss: 0.675741]
[Epoch 0/50] [Batch 5/59] [D loss: 0.692798] [G loss: 0.676392]
[Epoch 0/50] [Batch 6/59] [D loss: 0.692581] [G loss: 0.677068]
[Epoch 0/50] [Batch 7/59] [D loss: 0.692355] [G loss: 0.677738]
[Epoch 0/50] [Batch 8/59] [D loss: 0.692143] [G loss: 0.678277]
[Epoch 0/50] [Batch 9/59] [D loss: 0.691925] [G loss: 0.678694]
[Epoch 0/50] [Batch 10/59] [D loss: 0.691647] [G loss: 0.679031]
[Epoch 0/50] [Batch 11/59] [D loss: 0.691339] [G loss: 0.679034]
[Epoch 0/50] [Batch 12/59] [D loss: 0.691299] [G loss: 0.678465]
[Epoch 0/50] [Batch 13/59] [D loss: 0.691042] [G loss: 0.677697]
[Epoch 0/50] [Batch 14/59] [D loss: 0.691506] [G loss: 0.676357]
[Epoch 0/50] [Batch 15/59] [D loss:

#Displaying Generated image (Could be error as we need to run the training again)

In [ ]:
import matplotlib.pyplot as plt
out = generator(z)
n=607
f, axarr = plt.subplots(1,1)
z1 = z.cpu().detach().numpy()
out = out.cpu().detach().numpy()
axarr[1].imshow(out[n][0])

NameError: ignored